In [ ]:
!pip install benchmarx==0.0.11 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00


In [ ]:
# Imports
from benchmarx import Benchmark
from benchmarx.quadratic_problem import QuadraticProblem
import jax.numpy as jnp
from jax import random

In [ ]:
# Create Quadratic Problem

n = 5      # problem dimensionality

qp = QuadraticProblem(
    n=n,
    mineig=1,   # min eigenvalue
    maxeig=10   # max eigenvalue
)

# initial point
RNG = random.PRNGKey(1230)
RNG, _ = random.split(RNG)
x_init = random.normal(RNG, (n,))

In [ ]:
# Benchmark object to create an experiment
benchmark = Benchmark(
    problem=qp,
    runs=2,      # each method will be executed <runs> times
    methods = [
        {
            'GRADIENT_DESCENT_const_step': {
                'x_init' : x_init,
                'tol': 0,
                'maxiter': 300,
                'stepsize' : 2*1e-1,
                'acceleration': False,
                'label': 'GD_2e-1'
            },
            'GRADIENT_DESCENT_const_step2': {
                'x_init' : x_init,
                'tol': 0,
                'maxiter': 300,
                'stepsize' : 1e-2,
                'acceleration': False,
                'label': 'GD_1e-2'
            },
            'GRADIENT_DESCENT_adaptive_step': {
                'x_init' : x_init,
                'tol': 0,
                'maxiter': 300,
                'stepsize' : lambda iter_num:  1 / (iter_num + 9),
                'acceleration': False,
                'label': 'GD_1/(k+9)'
            }
        }
    ],
    metrics=[       # these metrics will be tracked as the experiment progresses
        "nit",
        "x",
        "f",
        "grad"
    ]
)

In [ ]:
# Run the experiment
res = benchmark.run()

In [ ]:
# Let's plot our results
res.plot(
    metrics= ['f', 'x_norm', 'f_gap', 'x_gap', 'grad_norm']
)